In [19]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

# --- Step 1: Read Annotator Data ---
# Replace these with your actual Excel file names.
df_a1 = pd.read_excel('annotator1.xlsx')
df_a2 = pd.read_excel('annotator2.xlsx')

# --- Step 1.1: Clean Column Names ---
df_a1.columns = df_a1.columns.str.strip()
df_a2.columns = df_a2.columns.str.strip()

# --- Step 1.2: Rename Columns if Needed ---
if 'Argument' not in df_a1.columns and 'Arguments' in df_a1.columns:
    df_a1.rename(columns={'Arguments': 'Argument'}, inplace=True)
if 'Argument' not in df_a2.columns and 'Arguments' in df_a2.columns:
    df_a2.rename(columns={'Arguments': 'Argument'}, inplace=True)

# --- Step 1.3: Preserve Original Order from Annotator1 ---
df_a1['order'] = df_a1.index

# --- Step 2: Select Only Relevant Columns ---
df_a1 = df_a1[['Argument', 'Sense', 'order']]
df_a2 = df_a2[['Argument', 'Sense']]

# Rename Sense columns to distinguish annotators.
df_a1.rename(columns={'Sense': 'Sense_a1'}, inplace=True)
df_a2.rename(columns={'Sense': 'Sense_a2'}, inplace=True)

# --- Step 3: Merge Data on "Argument" Field ---
merged_df = pd.merge(df_a1, df_a2, on='Argument', how='outer', sort=False)

# --- Step 4: Fill Missing "order" Values for Rows from Annotator2 Only ---
# For rows coming only from annotator2, assign them an order value greater than any in annotator1.
max_order = df_a1['order'].max() if not df_a1.empty else 0
merged_df['order'] = merged_df['order'].fillna(
    pd.Series(max_order + merged_df.index.to_series() + 1, index=merged_df.index)
)

# --- Step 5: Replace Missing Sense Values with "N/A" ---
merged_df['Sense_a1'] = merged_df['Sense_a1'].fillna("N/A")
merged_df['Sense_a2'] = merged_df['Sense_a2'].fillna("N/A")

# --- Step 6: Drop Rows Where Both Annotators Have "N/A" ---
merged_df = merged_df[~((merged_df['Sense_a1'] == "N/A") & (merged_df['Sense_a2'] == "N/A"))]

# --- Step 7: Compute Row-Level Agreement ---
# Compare sense labels in a case-insensitive manner (using lower() and strip()).
def compute_agreement(row):
    s1 = row['Sense_a1'].lower().strip()
    s2 = row['Sense_a2'].lower().strip()
    if s1 == "n/a" or s2 == "n/a":
        return 0
    return 1 if s1 == s2 else 0

merged_df['Agreement'] = merged_df.apply(compute_agreement, axis=1)

# --- Step 8: Sort by Original Order ---
merged_df = merged_df.sort_values(by='order').drop(columns=['order'])

# --- Step 9: Compute Global Metrics ---
# Consider only rows where both annotators provided a sense (i.e. not "N/A").
valid_df = merged_df[(merged_df['Sense_a1'] != "N/A") & (merged_df['Sense_a2'] != "N/A")]
y_true = valid_df['Sense_a1'].str.lower().str.strip()
y_pred = valid_df['Sense_a2'].str.lower().str.strip()

overall_agreement = merged_df['Agreement'].mean()
accuracy = accuracy_score(y_true, y_pred) if not valid_df.empty else 0
f1 = f1_score(y_true, y_pred, average='macro') if not valid_df.empty else 0
kappa = cohen_kappa_score(y_true, y_pred) if not valid_df.empty else 0

print("Global Agreement Metrics:")
print(f"  Overall Agreement (row-level): {overall_agreement:.2f}")
print(f"  Accuracy: {accuracy:.2f}")
print(f"  Macro F1 Score: {f1:.2f}")
print(f"  Cohen's Kappa: {kappa:.2f}")

# --- Step 10: Save the Comparison Results to an Excel File ---
output_filename = "Final_Result.xlsx"
merged_df.to_excel(output_filename, index=False)
print(f"Comparison results have been saved to '{output_filename}'")


Global Agreement Metrics:
  Overall Agreement (row-level): 0.42
  Accuracy: 0.94
  Macro F1 Score: 0.73
  Cohen's Kappa: 0.92
Comparison results have been saved to 'Final_Result.xlsx'


In [20]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

# --- Step 1: Read Annotator Data ---
# Replace these file names with your actual Excel file names.
df_a1 = pd.read_excel('annotator1.xlsx')
df_a2 = pd.read_excel('annotator2.xlsx')

# --- Step 1.1: Clean Column Names ---
df_a1.columns = df_a1.columns.str.strip()
df_a2.columns = df_a2.columns.str.strip()

# --- Step 1.2: Rename Columns if Needed ---
# For annotator 1:
if 'Argument' not in df_a1.columns and 'Arguments' in df_a1.columns:
    df_a1.rename(columns={'Arguments': 'Argument'}, inplace=True)
if 'Argument2' not in df_a1.columns and 'Arguments2' in df_a1.columns:
    df_a1.rename(columns={'Arguments2': 'Argument2'}, inplace=True)
if 'Sense' not in df_a1.columns and 'Senses' in df_a1.columns:
    df_a1.rename(columns={'Senses': 'Sense'}, inplace=True)
if 'Sense2' not in df_a1.columns and 'Senses2' in df_a1.columns:
    df_a1.rename(columns={'Senses2': 'Sense2'}, inplace=True)

# For annotator 2:
if 'Argument' not in df_a2.columns and 'Arguments' in df_a2.columns:
    df_a2.rename(columns={'Arguments': 'Argument'}, inplace=True)
if 'Argument2' not in df_a2.columns and 'Arguments2' in df_a2.columns:
    df_a2.rename(columns={'Arguments2': 'Argument2'}, inplace=True)
if 'Sense' not in df_a2.columns and 'Senses' in df_a2.columns:
    df_a2.rename(columns={'Senses': 'Sense'}, inplace=True)
if 'Sense2' not in df_a2.columns and 'Senses2' in df_a2.columns:
    df_a2.rename(columns={'Senses2': 'Sense2'}, inplace=True)

# --- Step 1.3: (Optional) Preserve Order from Annotator1 ---
df_a1['order'] = df_a1.index

# --- Step 2: Reshape Each Annotator's Data to Long Format ---
# Concatenate the two sets of columns (first and second relations) for each annotator.
df_a1_long = pd.DataFrame({
    "Argument": pd.concat([df_a1["Argument"], df_a1["Argument2"]], ignore_index=True),
    "Sense_a1": pd.concat([df_a1["Sense"], df_a1["Sense2"]], ignore_index=True)
})
df_a2_long = pd.DataFrame({
    "Argument": pd.concat([df_a2["Argument"], df_a2["Argument2"]], ignore_index=True),
    "Sense_a2": pd.concat([df_a2["Sense"], df_a2["Sense2"]], ignore_index=True)
})

# --- Step 3: Merge the Two Long DataFrames on "Argument" ---
merged_df = pd.merge(df_a1_long, df_a2_long, on="Argument", how="outer", sort=False)

# --- Step 4: Process Missing Values ---
# Fill missing sense values with "N/A"
merged_df["Sense_a1"] = merged_df["Sense_a1"].fillna("N/A")
merged_df["Sense_a2"] = merged_df["Sense_a2"].fillna("N/A")

# Drop rows where both annotators have no annotation (i.e. both are "N/A")
merged_df = merged_df[~((merged_df["Sense_a1"] == "N/A") & (merged_df["Sense_a2"] == "N/A"))]

# --- Step 5: Compute Row-Level Agreement ---
# Compare sense labels in a case-insensitive manner.
def compute_agreement(row):
    s1 = row["Sense_a1"].lower().strip()
    s2 = row["Sense_a2"].lower().strip()
    if s1 == "n/a" or s2 == "n/a":
        return 0
    return 1 if s1 == s2 else 0

merged_df["Agreement"] = merged_df.apply(compute_agreement, axis=1)

# --- Step 6: Compute Global Metrics ---
# Use only rows where both annotators provided a sense (i.e. not "N/A")
valid_df = merged_df[(merged_df["Sense_a1"] != "N/A") & (merged_df["Sense_a2"] != "N/A")]
y_true = valid_df["Sense_a1"].str.lower().str.strip()
y_pred = valid_df["Sense_a2"].str.lower().str.strip()

overall_agreement = merged_df["Agreement"].mean()
accuracy = accuracy_score(y_true, y_pred) if not valid_df.empty else 0
f1 = f1_score(y_true, y_pred, average="macro") if not valid_df.empty else 0
kappa = cohen_kappa_score(y_true, y_pred) if not valid_df.empty else 0

print("Global Agreement Metrics:")
print(f"  Overall Agreement (row-level): {overall_agreement:.2f}")
print(f"  Accuracy: {accuracy:.2f}")
print(f"  Macro F1 Score: {f1:.2f}")
print(f"  Cohen's Kappa: {kappa:.2f}")

# --- Step 7: Save the Final Merged Comparison to an Excel File ---
output_filename = "Retest.xlsx"
merged_df.to_excel(output_filename, index=False)
print(f"Comparison results have been saved to '{output_filename}'")


Global Agreement Metrics:
  Overall Agreement (row-level): 0.41
  Accuracy: 0.94
  Macro F1 Score: 0.73
  Cohen's Kappa: 0.93
Comparison results have been saved to 'Retest.xlsx'


In [25]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

# --- Step 1: Read Annotator Data ---
# Replace these file names with your actual Excel file names.
df_a1 = pd.read_excel('Annotator1_water_mainbreak.xlsx')
df_a2 = pd.read_excel('Annotator2_water_mainbreak.xlsx')

# --- Step 1.1: Clean Column Names ---
df_a1.columns = df_a1.columns.str.strip()
df_a2.columns = df_a2.columns.str.strip()

# --- Step 1.2: Rename Columns if Needed ---
# For annotator 1:
if 'Argument' not in df_a1.columns and 'Arguments' in df_a1.columns:
    df_a1.rename(columns={'Arguments': 'Argument'}, inplace=True)
if 'Argument2' not in df_a1.columns and 'Arguments2' in df_a1.columns:
    df_a1.rename(columns={'Arguments2': 'Argument2'}, inplace=True)
if 'Sense' not in df_a1.columns and 'Senses' in df_a1.columns:
    df_a1.rename(columns={'Senses': 'Sense'}, inplace=True)
if 'Sense2' not in df_a1.columns and 'Senses2' in df_a1.columns:
    df_a1.rename(columns={'Senses2': 'Sense2'}, inplace=True)

# For annotator 2:
if 'Argument' not in df_a2.columns and 'Arguments' in df_a2.columns:
    df_a2.rename(columns={'Arguments': 'Argument'}, inplace=True)
if 'Argument2' not in df_a2.columns and 'Arguments2' in df_a2.columns:
    df_a2.rename(columns={'Arguments2': 'Argument2'}, inplace=True)
if 'Sense' not in df_a2.columns and 'Senses' in df_a2.columns:
    df_a2.rename(columns={'Senses': 'Sense'}, inplace=True)
if 'Sense2' not in df_a2.columns and 'Senses2' in df_a2.columns:
    df_a2.rename(columns={'Senses2': 'Sense2'}, inplace=True)

# --- Step 1.3: (Optional) Preserve Order from Annotator1 ---
df_a1['order'] = df_a1.index

# --- Step 2: Reshape Each Annotator's Data to Long Format ---
# Concatenate the two sets of columns (first and second relations) for each annotator.
df_a1_long = pd.DataFrame({
    "Argument": pd.concat([df_a1["Argument"], df_a1["Argument2"]], ignore_index=True),
    "Sense_a1": pd.concat([df_a1["Sense"], df_a1["Sense2"]], ignore_index=True)
})
df_a2_long = pd.DataFrame({
    "Argument": pd.concat([df_a2["Argument"], df_a2["Argument2"]], ignore_index=True),
    "Sense_a2": pd.concat([df_a2["Sense"], df_a2["Sense2"]], ignore_index=True)
})

# --- Step 3: Merge the Two Long DataFrames on "Argument" ---
merged_df = pd.merge(df_a1_long, df_a2_long, on="Argument", how="outer", sort=False)

# --- Step 4: Process Missing Values ---
# Fill missing sense values with "N/A"
merged_df["Sense_a1"] = merged_df["Sense_a1"].fillna("N/A")
merged_df["Sense_a2"] = merged_df["Sense_a2"].fillna("N/A")

# Drop rows where both annotators have no annotation (i.e. both are "N/A")
merged_df = merged_df[~((merged_df["Sense_a1"] == "N/A") & (merged_df["Sense_a2"] == "N/A"))]

# --- Step 5: Compute Row-Level Agreement ---
# Compare sense labels in a case-insensitive manner.
def compute_agreement(row):
    s1 = row["Sense_a1"].lower().strip()
    s2 = row["Sense_a2"].lower().strip()
    if s1 == "n/a" or s2 == "n/a":
        return 0
    return 1 if s1 == s2 else 0

merged_df["Agreement"] = merged_df.apply(compute_agreement, axis=1)

# --- Step 6: Compute Global Metrics ---
# Use only rows where both annotators provided a sense (i.e. not "N/A")
valid_df = merged_df[(merged_df["Sense_a1"] != "N/A") & (merged_df["Sense_a2"] != "N/A")]
y_true = valid_df["Sense_a1"].str.lower().str.strip()
y_pred = valid_df["Sense_a2"].str.lower().str.strip()

overall_agreement = merged_df["Agreement"].mean()
accuracy = accuracy_score(y_true, y_pred) if not valid_df.empty else 0
f1 = f1_score(y_true, y_pred, average="macro") if not valid_df.empty else 0
kappa = cohen_kappa_score(y_true, y_pred) if not valid_df.empty else 0

print("Global Agreement Metrics:")
print(f"  Overall Agreement (row-level): {overall_agreement:.2f}")
print(f"  Accuracy: {accuracy:.2f}")
print(f"  Macro F1 Score: {f1:.2f}")
print(f"  Cohen's Kappa: {kappa:.2f}")

# --- Step 7: Save the Final Merged Comparison to an Excel File ---
output_filename = "water_mainbreak_agreement.xlsx"
merged_df.to_excel(output_filename, index=False)
print(f"Comparison results have been saved to '{output_filename}'")


Global Agreement Metrics:
  Overall Agreement (row-level): 0.62
  Accuracy: 0.98
  Macro F1 Score: 0.79
  Cohen's Kappa: 0.97
Comparison results have been saved to 'water_mainbreak_agreement.xlsx'
